In [10]:
import mcp.types 
print(dir(mcp.types))


['Annotated', 'Annotations', 'Any', 'AnyFunction', 'AnyUrl', 'AudioContent', 'BaseMetadata', 'BaseModel', 'BlobResourceContents', 'CONNECTION_CLOSED', 'CallToolRequest', 'CallToolRequestParams', 'CallToolResult', 'Callable', 'CancelledNotification', 'CancelledNotificationParams', 'ClientCapabilities', 'ClientNotification', 'ClientRequest', 'ClientResult', 'CompleteRequest', 'CompleteRequestParams', 'CompleteResult', 'Completion', 'CompletionArgument', 'CompletionContext', 'CompletionsCapability', 'ConfigDict', 'Content', 'ContentBlock', 'CreateMessageRequest', 'CreateMessageRequestParams', 'CreateMessageResult', 'Cursor', 'DEFAULT_NEGOTIATED_VERSION', 'ElicitRequest', 'ElicitRequestParams', 'ElicitRequestedSchema', 'ElicitResult', 'ElicitationCapability', 'EmbeddedResource', 'EmptyResult', 'ErrorData', 'Field', 'FileUrl', 'Generic', 'GetPromptRequest', 'GetPromptRequestParams', 'GetPromptResult', 'INTERNAL_ERROR', 'INVALID_PARAMS', 'INVALID_REQUEST', 'ImageContent', 'Implementation', '

In [15]:
from mcp.types import ProgressNotification
ProgressNotification()

ValidationError: 2 validation errors for ProgressNotification
method
  Field required [type=missing, input_value={}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.11/v/missing
params
  Field required [type=missing, input_value={}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.11/v/missing

In [ ]:
pip install "mcp[cli]" 

Note: you may need to restart the kernel to use updated packages.


In [4]:
#!/usr/bin/env python3
"""
CAD Geometry MCP Server
A Model Context Protocol server that wraps CadQuery primitives for CAD generation.
"""

import json
import tempfile
import os
from typing import Dict, Any, Optional, List
from pathlib import Path
import cadquery as cq
from cadquery import exporters
# MCP imports
from mcp.server.models import InitializationOptions
from mcp.server import Server
from mcp.server.stdio import stdio_server
from mcp.types import (
    Resource,
    Tool,
    TextContent,
    ImageContent,
    EmbeddedResource,
)

# CAD imports
import cadquery as cq
from cadquery import exporters

# Design Patterns Implementation
class CADModelBuilder:
    """Builder pattern for CAD model construction"""
    
    def __init__(self):
        self.workplane = cq.Workplane("XY")
        self.operations = []
        
    def add_box(self, length: float, width: float, height: float) -> 'CADModelBuilder':
        self.operations.append(("box", {"length": length, "width": width, "height": height}))
        self.workplane = self.workplane.box(length, width, height)
        return self
        
    def add_cylinder(self, radius: float, height: float) -> 'CADModelBuilder':
        self.operations.append(("cylinder", {"radius": radius, "height": height}))
        self.workplane = self.workplane.cylinder(height, radius)
        return self
        
    def add_hole(self, diameter: float, depth: Optional[float] = None) -> 'CADModelBuilder':
        self.operations.append(("hole", {"diameter": diameter, "depth": depth}))
        if depth:
            self.workplane = self.workplane.hole(diameter, depth)
        else:
            self.workplane = self.workplane.hole(diameter)
        return self
        
    def add_fillet(self, radius: float) -> 'CADModelBuilder':
        self.operations.append(("fillet", {"radius": radius}))
        self.workplane = self.workplane.edges().fillet(radius)
        return self
        
    def build(self) -> cq.Workplane:
        return self.workplane

class CADExporter:
    """Strategy pattern for different export formats"""
    
    @staticmethod
    def export_stl(model: cq.Workplane, filepath: str) -> bool:
        try:
            exporters.export(model, filepath, exportType=exporters.ExportTypes.STL)
            return True
        except Exception as e:
            print(f"STL export failed: {e}")
            return False
            
    @staticmethod
    def export_step(model: cq.Workplane, filepath: str) -> bool:
        try:
            exporters.export(model, filepath, exportType=exporters.ExportTypes.STEP)
            return True
        except Exception as e:
            print(f"STEP export failed: {e}")
            return False
            
    @staticmethod
    def export_svg(model: cq.Workplane, filepath: str) -> bool:
        try:
            exporters.export(model, filepath, exportType=exporters.ExportTypes.SVG)
            return True
        except Exception as e:
            print(f"SVG export failed: {e}")
            return False

class CADModelCache:
    """Flyweight pattern for caching common parts"""
    
    _cache: Dict[str, cq.Workplane] = {}
    
    @classmethod
    def get_model(cls, key: str) -> Optional[cq.Workplane]:
        return cls._cache.get(key)
        
    @classmethod
    def cache_model(cls, key: str, model: cq.Workplane) -> None:
        cls._cache[key] = model
        
    @classmethod
    def clear_cache(cls) -> None:
        cls._cache.clear()

# Global model storage
current_models: Dict[str, cq.Workplane] = {}
temp_dir = tempfile.mkdtemp()

app = Server("cad-geometry-server")

@app.list_tools()
async def handle_list_tools() -> List[Tool]:
    """List available CAD tools"""
    return [
        Tool(
            name="create_box",
            description="Create a rectangular box/cube",
            inputSchema={
                "type": "object",
                "properties": {
                    "name": {"type": "string", "description": "Model name/identifier"},
                    "length": {"type": "number", "description": "Length (X dimension)"},
                    "width": {"type": "number", "description": "Width (Y dimension)"},
                    "height": {"type": "number", "description": "Height (Z dimension)"},
                },
                "required": ["name", "length", "width", "height"],
            },
        ),
        Tool(
            name="create_cylinder",
            description="Create a cylinder",
            inputSchema={
                "type": "object",
                "properties": {
                    "name": {"type": "string", "description": "Model name/identifier"},
                    "radius": {"type": "number", "description": "Cylinder radius"},
                    "height": {"type": "number", "description": "Cylinder height"},
                },
                "required": ["name", "radius", "height"],
            },
        ),
        Tool(
            name="create_flange",
            description="Create a standard flange with bolt holes",
            inputSchema={
                "type": "object",
                "properties": {
                    "name": {"type": "string", "description": "Model name/identifier"},
                    "outer_diameter": {"type": "number", "description": "Outer diameter of flange"},
                    "inner_diameter": {"type": "number", "description": "Inner diameter (bore)"},
                    "thickness": {"type": "number", "description": "Flange thickness"},
                    "bolt_circle_diameter": {"type": "number", "description": "Bolt circle diameter"},
                    "bolt_hole_diameter": {"type": "number", "description": "Bolt hole diameter"},
                    "num_bolts": {"type": "integer", "description": "Number of bolt holes", "default": 6},
                },
                "required": ["name", "outer_diameter", "inner_diameter", "thickness", "bolt_circle_diameter", "bolt_hole_diameter"],
            },
        ),
        Tool(
            name="apply_fillet",
            description="Apply fillet to edges of existing model",
            inputSchema={
                "type": "object",
                "properties": {
                    "name": {"type": "string", "description": "Model name to modify"},
                    "radius": {"type": "number", "description": "Fillet radius"},
                },
                "required": ["name", "radius"],
            },
        ),
        Tool(
            name="export_model",
            description="Export model to file",
            inputSchema={
                "type": "object",
                "properties": {
                    "name": {"type": "string", "description": "Model name to export"},
                    "format": {"type": "string", "enum": ["stl", "step", "svg"], "description": "Export format"},
                    "filename": {"type": "string", "description": "Output filename (optional)"},
                },
                "required": ["name", "format"],
            },
        ),
        Tool(
            name="get_mass_properties",
            description="Get mass properties of a model",
            inputSchema={
                "type": "object",
                "properties": {
                    "name": {"type": "string", "description": "Model name"},
                },
                "required": ["name"],
            },
        ),
        Tool(
            name="list_models",
            description="List all current models in memory",
            inputSchema={
                "type": "object",
                "properties": {},
            },
        ),
    ]

@app.call_tool()
async def handle_call_tool(name: str, arguments: Dict[str, Any]) -> List[TextContent]:
    """Handle tool calls"""
    
    try:
        if name == "create_box":
            model_name = arguments["name"]
            length = float(arguments["length"])
            width = float(arguments["width"])
            height = float(arguments["height"])
            
            # Check cache first
            cache_key = f"box_{length}_{width}_{height}"
            cached_model = CADModelCache.get_model(cache_key)
            
            if cached_model:
                current_models[model_name] = cached_model
                result = f"Created box '{model_name}' from cache: {length} x {width} x {height}"
            else:
                builder = CADModelBuilder()
                model = builder.add_box(length, width, height).build()
                current_models[model_name] = model
                CADModelCache.cache_model(cache_key, model)
                result = f"Created box '{model_name}': {length} x {width} x {height}"
            
            return [TextContent(type="text", text=result)]
            
        elif name == "create_cylinder":
            model_name = arguments["name"]
            radius = float(arguments["radius"])
            height = float(arguments["height"])
            
            cache_key = f"cylinder_{radius}_{height}"
            cached_model = CADModelCache.get_model(cache_key)
            
            if cached_model:
                current_models[model_name] = cached_model
                result = f"Created cylinder '{model_name}' from cache: r={radius}, h={height}"
            else:
                builder = CADModelBuilder()
                model = builder.add_cylinder(radius, height).build()
                current_models[model_name] = model
                CADModelCache.cache_model(cache_key, model)
                result = f"Created cylinder '{model_name}': r={radius}, h={height}"
            
            return [TextContent(type="text", text=result)]
            
        elif name == "create_flange":
            model_name = arguments["name"]
            outer_d = float(arguments["outer_diameter"])
            inner_d = float(arguments["inner_diameter"])
            thickness = float(arguments["thickness"])
            bolt_circle_d = float(arguments["bolt_circle_diameter"])
            bolt_hole_d = float(arguments["bolt_hole_diameter"])
            num_bolts = int(arguments.get("num_bolts", 6))
            
            # Validation
            if inner_d >= outer_d:
                return [TextContent(type="text", text="Error: Inner diameter must be less than outer diameter")]
            if bolt_circle_d >= outer_d:
                return [TextContent(type="text", text="Error: Bolt circle diameter must be less than outer diameter")]
                
            # Create flange
            flange = (cq.Workplane("XY")
                     .circle(outer_d/2)
                     .circle(inner_d/2)  # Inner hole
                     .extrude(thickness)
                     .faces(">Z")  # Top face
                     .workplane()
                     .polarArray(bolt_circle_d/2, 0, 360, num_bolts)
                     .circle(bolt_hole_d/2)
                     .cutThruAll())
            
            current_models[model_name] = flange
            result = f"Created flange '{model_name}': OD={outer_d}, ID={inner_d}, thickness={thickness}, {num_bolts} bolt holes"
            
            return [TextContent(type="text", text=result)]
            
        elif name == "apply_fillet":
            model_name = arguments["name"]
            radius = float(arguments["radius"])
            
            if model_name not in current_models:
                return [TextContent(type="text", text=f"Error: Model '{model_name}' not found")]
                
            try:
                model = current_models[model_name]
                filleted_model = model.edges().fillet(radius)
                current_models[model_name] = filleted_model
                result = f"Applied fillet r={radius} to model '{model_name}'"
                return [TextContent(type="text", text=result)]
            except Exception as e:
                return [TextContent(type="text", text=f"Error applying fillet: {str(e)}")]
                
        elif name == "export_model":
            model_name = arguments["name"]
            export_format = arguments["format"].lower()
            filename = arguments.get("filename", f"{model_name}.{export_format}")
            
            if model_name not in current_models:
                return [TextContent(type="text", text=f"Error: Model '{model_name}' not found")]
                
            filepath = os.path.join(temp_dir, filename)
            model = current_models[model_name]
            
            success = False
            if export_format == "stl":
                success = CADExporter.export_stl(model, filepath)
            elif export_format == "step":
                success = CADExporter.export_step(model, filepath)
            elif export_format == "svg":
                success = CADExporter.export_svg(model, filepath)
                
            if success:
                result = f"Exported '{model_name}' to {filepath}"
            else:
                result = f"Failed to export '{model_name}' as {export_format}"
                
            return [TextContent(type="text", text=result)]
            
        elif name == "get_mass_properties":
            model_name = arguments["name"]
            
            if model_name not in current_models:
                return [TextContent(type="text", text=f"Error: Model '{model_name}' not found")]
                
            model = current_models[model_name]
            
            # Get bounding box and volume
            bb = model.val().BoundingBox()
            volume = model.val().Volume()
            
            result = {
                "model": model_name,
                "volume": volume,
                "bounding_box": {
                    "min": [bb.xmin, bb.ymin, bb.zmin],
                    "max": [bb.xmax, bb.ymax, bb.zmax],
                    "dimensions": [bb.xmax - bb.xmin, bb.ymax - bb.ymin, bb.zmax - bb.zmin]
                }
            }
            
            return [TextContent(type="text", text=json.dumps(result, indent=2))]
            
        elif name == "list_models":
            if not current_models:
                return [TextContent(type="text", text="No models currently loaded")]
                
            model_list = []
            for name in current_models.keys():
                model_list.append(f"- {name}")
                
            result = f"Current models:\n" + "\n".join(model_list)
            return [TextContent(type="text", text=result)]
            
        else:
            return [TextContent(type="text", text=f"Unknown tool: {name}")]
            
    except Exception as e:
        return [TextContent(type="text", text=f"Error executing {name}: {str(e)}")]

async def main():
    # Run the MCP server
    async with stdio_server() as (read_stream, write_stream):
        await app.run(
            read_stream,
            write_stream,
            InitializationOptions(
                server_name="cad-geometry-server",
                server_version="0.1.0",
                capabilities=app.get_capabilities(
                    notification_options=None,
                    experimental_capabilities=None,
                ),
            ),
        )

import nest_asyncio, asyncio
nest_asyncio.apply()

await main()


# if __name__ == "__main__":
#     import asyncio
#     asyncio.run(main())

AttributeError: 'OutStream' object has no attribute 'buffer'